In [ ]:
import requests
import os
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray as xr
import xarray_leaflet
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ipyleaflet import Map, basemaps

In [ ]:
fixed_north = {
    'name': 'EPSG:3413',
    'custom': True, 
    'proj4def': '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs',
    'origin': [-4194304, 4194304],
    'bounds': [
        [-4194304, -4194304],
        [4194304, 4194304]   
    ],
    'resolutions': [
        16384.0,
        8192.0,
        4096.0,
        2048.0,
        1024.0,
        512.0,
        256.0
    ]
}

In [ ]:
cmap = plt.cm.get_cmap('jet', 256)
newcolors = cmap(np.linspace(0, 1, 256))
newcolors[:,-1][0:30] = 0  # transparent for low values
cmap = ListedColormap(newcolors)

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/eu_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = name + '/' + name + '/w001001.adf'

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [ ]:
da = xr.open_rasterio(adffile)
da

In [ ]:
crs = da.rio.crs
nan = da.attrs['nodatavals'][0]
da = da.sel(band=1)
da = xr.where(da==nan, np.nan, da)
da = da.rio.write_crs(crs)

In [ ]:
m = Map(center=(64.47601699176504, -40.652543092440325),
        zoom=1,
        basemap=basemaps.NASAGIBS.BlueMarble3413,
        crs=fixed_north,
       interpolation='nearest')
m

In [ ]:
l = da.leaflet.plot(m, x_dim='x', y_dim='y', transform2=xarray_leaflet.transform.passthrough, colormap=cmap)
l.interact(opacity=(0.0,1.0,0.1))